# The hour of λ

## Algebraic data types

We can understand _types_ as collections of *values*. For instance:

In [1]:
val i: Int = 1
val j: Int = 0
val c: Char = 'a'
val d: Char = 'λ'
val s: String = "lambda"
val t: String = "pi"
val b: Boolean = true
val b2: Boolean = false

i: Int = 1
j: Int = 0
c: Char = 'a'
d: Char = '\u03bb'
s: String = "lambda"
t: String = "pi"
b: Boolean = true
b2: Boolean = false

These are *primitive* types, but we will normally want to create our own types, carefully crafted to represent the particular features of our application domain. We do that by combining types using so-called algebraic data type constructors: *sums* and *products*. 

### Product types

Product types (more precisely, _record types_) are declared in Scala using so-called *case classes*:

In [2]:
case class Circle(radius: Int)
case class Rectangle(width: Int, height: Int)
case class Triangle(base: Int) 

defined class Circle
defined class Rectangle
defined class Triangle

We extract the information encoded by records using their named attributes: 

In [3]:
val i: Int = Circle(3).radius
val w: Int = Rectangle(1,2).width

i: Int = 3
w: Int = 1

### Sum types

Sum types encode values taken from the disjoint union of other types. For instance, a geometrical shape may be defined as the sum of circles, rectangles and triangles: 

In [4]:
abstract class Shape
case class Circle(radius: Int) extends Shape
case class Rectangle(width: Int, height: Int) extends Shape
case class Triangle(base: Int) extends Shape

defined class Shape
defined class Circle
defined class Rectangle
defined class Triangle

We redefined the `Circle`, `Rectangle` and `Triangle` classes for convenience, but, in general, pre-existing definitions would be injected in the sum type by their own constructor. In this new form, when we create a `Rectangle` we are creating right away a `Shape` value:  

In [5]:
val shape: Shape = Rectangle(5,4)

shape: Shape = Rectangle(5, 4)

To extract information from a value of sum type, we use *pattern matching*: 

In [6]:
val shapeType: String = shape match {
    case Circle(_) => "circle"
    case Rectangle(_, _) => "rectangle"
    case Triangle(_) => "triangle"
}

shapeType: String = "rectangle"

### Function types

Functions are computational devices that transform input values into output values, and do nothing else (i.e. they are free of side-effects). Function type values are also called *lambda expressions*. For instance:

In [7]:
val rectangleArea: Rectangle => Int = 
    (r: Rectangle) => r.width * r.height

rectangleArea: Rectangle => Int = ammonite.$sess.cmd6$Helper$$Lambda$2271/1402424841@421a6c8f

In [8]:
import scala.math.{Pi, pow}

val cicleArea: Circle => Double = 
    (c: Circle) => Pi * pow(c.radius, 2)

import scala.math.{Pi, pow}


cicleArea: Circle => Double = ammonite.$sess.cmd7$Helper$$Lambda$2308/1440279310@53045394

In [9]:
val triangleArea: Triangle => Double = 
    (t: Triangle) => t.base * t.base / 2.0

triangleArea: Triangle => Double = ammonite.$sess.cmd8$Helper$$Lambda$2313/1055094051@1189c571

In [10]:
val area: Shape => Double = (shape: Shape) => 
    shape match {
        case c: Circle => cicleArea(c)
        case r: Rectangle => rectangleArea(r)
        case t: Triangle => triangleArea(t)
    }

area: Shape => Double = ammonite.$sess.cmd9$Helper$$Lambda$2321/890623627@d3db356

We _extract_ information from a function value by applying it to a given input value:

In [11]:
area.apply(Rectangle(5,4))
// In Scala, we can omit the `apply` method
area(Circle(1))
area(Triangle(1))

res10_0: Double = 20.0
res10_1: Double = 3.141592653589793
res10_2: Double = 0.5

In Scala, functions are more commonly written as *methods*, for instance:

In [12]:
def area(shape: Shape): Double = 
    shape match {
        case c: Circle => cicleArea(c)
        case r: Rectangle => rectangleArea(r)
        case t: Triangle => triangleArea(t)
    }

defined function area

## Recursive types

Recursive types are types which, somehow, refer to themselves in their own definition. For example, a list can be the empty list (`Nil`) or a product consisting of a head and a tail (`Cons`), a list itself:

In [13]:
sealed abstract class ShapeList
case class Nil() extends ShapeList
case class Cons(head: Shape, tail: ShapeList) extends ShapeList

defined class ShapeList
defined class Nil
defined class Cons

In [14]:
val list: ShapeList = Cons(Circle(1), Cons(Rectangle(4,5), Cons(Triangle(4), Nil())))

list: ShapeList = Cons(
  Circle(1),
  Cons(Rectangle(4, 5), Cons(Triangle(4), Nil()))
)

### Functions over lists, imperatively

We can implement functions over lists in an imperative style using variables and while loops:

In [15]:
def sumAreasI(list: ShapeList): Double = {
    var result: Double = 0.0
    var cur: ShapeList = list 
    while (cur != Nil()){
        val head: Shape = cur.asInstanceOf[Cons].head
        result = result + area(head)
        cur = cur.asInstanceOf[Cons].tail 
    }
    result
}

defined function sumAreasI

In [16]:
sumAreasI(list)

res15: Double = 31.141592653589793

### Functions over lists, recursively (i.e. functionally)

However, this is not the functional style. In functional programming, we commonly avoid `var`s (mutable variables) and use recursion instead of conventional loops:

In [17]:
def sumAreas(list: ShapeList): Double = 
    list match {
        case Nil() => 0.0
        case Cons(head, tail) => area(head) + sumAreas(tail)
    }

defined function sumAreas

In [18]:
sumAreas(list)

res17: Double = 31.141592653589793

### Exercise

In [19]:
def perimeter(shape: Shape): Double =
    shape match {
        case Rectangle(w, h) => 2* (w + h)
        case Circle(r) => 2 * Pi * r
        case Triangle(b) => 3 * b
    }

defined function perimeter

In [20]:
def sumPerimeters(list: ShapeList): Double = 
    list match {
        case Nil() => 0.0
        case Cons(head, tail) => perimeter(head) + sumPerimeters(tail)
    } 

defined function sumPerimeters

In [21]:
sumPerimeters(list)

res20: Double = 36.283185307179586

## Modularisation

As you surely noticed, the functions `sumPerimeters` and `sumAreas` are like two drops of water. How could we achieve a more modular definition, reusing what they have in common? Functional programming excels at modularity, and offers different techniques to this aim. Here, we will review two of them: higher-order functions, parametric polymorphism and type classes.

### Higher-order functions

First, the difference between both functions lies in the particular function being applied to each shape (`area` or `perimeter`). We can abstract away this different by adding a new function parameters. The new parameter being a function, we end up with a function that receives functions. This is called a higher-order function.

In [22]:
def applyFunctionOnShapes(f: Shape => Double)(list: ShapeList): Double = 
    list match {
        case Nil() => 0.0
        case Cons(head, tail) => f(head) + applyFunctionOnShapes(f)(tail)
    }

defined function applyFunctionOnShapes

We can now redefine our functions in a more modular way, by composing the generic module `applyFunctionOnShapes` and the specific one `perimeter` or `area`.

In [23]:
val sumPerimeters: ShapeList => Double = 
    applyFunctionOnShapes(perimeter)

sumPerimeters: ShapeList => Double = ammonite.$sess.cmd22$Helper$$Lambda$2402/1488170144@79244f10

In [24]:
val sumAreas: ShapeList => Double = 
    applyFunctionOnShapes(area)

sumAreas: ShapeList => Double = ammonite.$sess.cmd23$Helper$$Lambda$2407/1501344579@5fc605f1

### Parametric polymorphism

Our definition of lists is way too specific. What if we need a list of integers? We would essentially copy-&-paste that definition and substitute `Int` for `Shape` everywhere. Instead, we can offer a _generic_ definition as follows:

In [25]:
sealed abstract class List[A]
case class Nil[A]() extends List[A]
case class Cons[A](head: A, tail: List[A]) extends List[A]

defined class List
defined class Nil
defined class Cons

In [26]:
val list: List[Shape] = Cons(Circle(1), Cons(Rectangle(4,5), Cons(Triangle(4), Nil())))

list: List[Shape] = Cons(
  Circle(1),
  Cons(Rectangle(4, 5), Cons(Triangle(4), Nil()))
)

How does this affect our generic `applyFunctionOnList`? Can we arrive to a generic definition as well? Yes, and this function is called `foldRight` in the functional programming literature:

In [27]:
def foldRight[A, B, C](b: B)(f: (A, B) => B)(list: List[A]): B = 
    list match {
        case Nil() => b
        case Cons(head, tail) => f(head, foldRight(b)(f)(tail))
    }

defined function foldRight

The modularisations have to be adapted as follows:

In [28]:
val sumAreas: List[Shape] => Double = 
    foldRight(0.0)((s: Shape, d: Double) => area(s) + d)

sumAreas: List[Shape] => Double = ammonite.$sess.cmd27$Helper$$Lambda$2440/1660206941@dc7dba0

In [29]:
val sumPerimeters: List[Shape] => Double = 
    foldRight(0.0)((s: Shape, d: Double) => perimeter(s) + d)

sumPerimeters: List[Shape] => Double = ammonite.$sess.cmd28$Helper$$Lambda$2446/331780121@6b1dd65f

### Higher-order functions (again)

In this section we will offer a different modularisation which attempts to grasp more faithfully the kind of computation that we want to perform. The problem with `foldRight` is that it's too generic, and our problem can be solved more meaningfully with a more specfic, yet generic, pattern. In particular, this pattern consists in two steps: first, map the list values using a function, and then, fold over that list of values. The resulting function is called `foldMap`:

In [30]:
def foldMap[A, B](zero: B, combine: (B, B) => B)(
        map: A => B)(list: List[A]): B = 
    list match {
        case Nil() => zero
        case Cons(head, tail) => 
            combine(map(head), foldMap(zero, combine)(map)(tail))
    }

defined function foldMap

or using `foldRight`:

In [31]:
def foldMap[A, B](zero: B)(combine: (B, B) => B)(map: A => B)(list: List[A]): B = 
    foldRight(zero)((a: A, b: B) => combine(map(a), b))(list)

defined function foldMap

But we may also give a different implementation of `foldMap`, which aims at exploting the paralelism of our processors or clusters. This is what essentially do Big data frameworks like MapReduce or Spark.

In [32]:
val sumAreas: List[Shape] => Double =
    foldMap(0.0)(_ + _)(area)

sumAreas: List[Shape] => Double = ammonite.$sess.cmd31$Helper$$Lambda$2453/1414409329@2b65d0dc

In [33]:
val sumPerimeters: List[Shape] => Double = 
    foldMap(0.0)(_ + _)(perimeter)

sumPerimeters: List[Shape] => Double = ammonite.$sess.cmd32$Helper$$Lambda$2460/371831125@20071608

### Type classes

In this section, we will adapt the previous modularization by using so-called *type classes*, a popular modularization technique within the functional programming community which was first used in the Haskell programming language. 

Basically, a type class defines that: a class of types for which we can given implementations of a number of functions. Type classes and algebras match very well. For instance, a semigroup is defined by a type A together with an implementation of the `combine` function, which must be associative. An a monoid is a semigroup which also provides a `zero`, i.e. a neutral element of the combine operation. In Scala:

In [47]:
trait Semigroup[A]{
    def combine(a1: A, a2: A): A
}

trait Monoid[A] extends Semigroup[A]{
    def zero: A
}

object Monoid{
    def apply[A](implicit M: Monoid[A]): Monoid[A] = M 
    
    implicit val DoubleIsAMonoid = new Monoid[Double]{
        def zero: Double = 0.0
        def combine(a1: Double, a2: Double): Double = a1 + a2
    }
}

defined trait Semigroup
defined trait Monoid
defined object Monoid

Now, we can express more succintly the signature of `foldMap` as follows:

In [49]:
def foldMap[A, B: Monoid](map: A => B)(list: List[A]): B = 
    foldRight(Monoid[B].zero)(
        (a: A, b: B) => Monoid[B].combine(map(a), b))(list)

defined function foldMap

and exploit the implicit mechanism of Scala to make calls without bothering to pass the zero and combine operations:

In [50]:
val sumAreas: List[Shape] => Double = 
    foldMap(area)

sumAreas: List[Shape] => Double = ammonite.$sess.cmd49$Helper$$Lambda$2540/1289812685@2926f145

In [51]:
def sumPerimeters: List[Shape] => Double = 
    foldMap(perimeter)

defined function sumPerimeters

In [52]:
sumPerimeters(list)

res51: Double = 36.283185307179586

What do type classes offer to us? Just the possibility of writing more concise definitions? Not only that: they enhance understanding, testability, correctness, evolvability, etc.